Project Description: Twitter US Airline Sentiment

Objective:
To implement the techniques learnt as a part of the course.

1. Import the libraries, load dataset, print shape of data, data description

In [1]:
import re, string, unicodedata
import pandas as pd
import nltk           

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup                 # Beautiful soup is a parsing library that can use different parsers.
from nltk import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Download data
tweet = pd.read_csv("/Users/USER/Downloads/Tweets.csv")


In [3]:
#Print shape
tweet.shape

(14640, 15)

In [4]:
#Print description
tweet.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


2. Understand of data-columns: 

In [5]:
#Drop all other columns except “text” and “airline_sentiment”
tweet=tweet[['text','airline_sentiment']]


In [6]:
#Check the shape of data
tweet.shape

(14640, 2)

In [7]:
#Check the shape of data.
tweet.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


3. Text pre-processing: Data preparation

In [8]:
#HTML tag, hashtag, number removal, special characters and punctuation removal
tweet['text']=tweet['text'].str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([0-9])","")
#Tokenization
tweet['text']=tweet['text'].apply(nltk.word_tokenize)
tweet.head()

,text,airline_sentiment
0,"[What, said]",neutral
1,"[plus, youve, added, commercials, to, the, exp...",positive
2,"[I, didnt, today, Must, mean, I, need, to, tak...",neutral
3,"[its, really, aggressive, to, blast, obnoxious...",negative
4,"[and, its, a, really, big, bad, thing, about, it]",negative


In [9]:
##Chose Stemming
stemmer = SnowballStemmer('english')
tweet['text']=tweet['text'].apply(lambda x: [stemmer.stem(y) for y in x])
tweet.head()

,text,airline_sentiment
0,"[what, said]",neutral
1,"[plus, youv, ad, commerci, to, the, experi, ta...",positive
2,"[i, didnt, today, must, mean, i, need, to, tak...",neutral
3,"[it, realli, aggress, to, blast, obnoxi, enter...",negative
4,"[and, it, a, realli, big, bad, thing, about, it]",negative


In [10]:
tweet['text'] = tweet['text'].str.join(" ")
tweet.head()



,text,airline_sentiment
0,what said,neutral
1,plus youv ad commerci to the experi tacki,positive
2,i didnt today must mean i need to take anoth trip,neutral
3,it realli aggress to blast obnoxi entertain in...,negative
4,and it a realli big bad thing about it,negative


In [11]:
##Splitting into train and test datasets:
X = tweet['text']
y = tweet['airline_sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


4. Vectorization

In [12]:
##CounVectorizer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() 
X_tra = vectorizer.fit_transform(X_train)
X_tes = vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb = MultinomialNB()
# train the model using X_train_dtm
nb.fit(X_tra, y_train)
# make class predictions for X_te
y_pred = nb.predict(X_tes)
##Accuracy
score = metrics.accuracy_score(y_test, y_pred)
print (score)
##CM
metrics.confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test, y_pred))

0.7721311475409836
              precision    recall  f1-score   support

    negative       0.77      0.95      0.85      2291
     neutral       0.72      0.40      0.51       774
    positive       0.82      0.56      0.66       595

    accuracy                           0.77      3660
   macro avg       0.77      0.64      0.68      3660
weighted avg       0.77      0.77      0.75      3660



In [14]:
#Using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_tra = vectorizer.fit_transform(X_train)
X_tes = vectorizer.transform(X_test)

from sklearn import metrics
mnb = MultinomialNB()
mnb.fit(X_tra, y_train)
y_pred = nb.predict(X_tes)
##Accuracy
score = metrics.accuracy_score(y_test, y_pred)
print (score)
##CM
metrics.confusion_matrix(y_test, y_pred)
print(metrics.classification_report(y_test, y_pred))

0.7341530054644809
              precision    recall  f1-score   support

    negative       0.73      0.98      0.83      2291
     neutral       0.75      0.29      0.42       774
    positive       0.82      0.37      0.51       595

    accuracy                           0.73      3660
   macro avg       0.76      0.55      0.59      3660
weighted avg       0.74      0.73      0.69      3660



Conclusion:
    
    By using different vectorizer (CountVectorizer and TfidfVectorizer) we achive different results (see classification reports above);
    By utilizing various preprocessing methods, we achieve higher accuracy in both training and test datasets
    I was not able to figure out contractions (had multiple fails when installing it), but I feel like if I could make it work, my 
        accuracy would go up as well
    I tried to make def tweet_to_words deifinition by including all preprocessing steps as part of tweet_to_words fuction, but in the end
        I could not figure out how to avoid list error, this would make the code more eloquent;
    I found that there are a couple of limitations such as when there is a link instead of text: if we could click on it we would found either an 
        an image or retweet or video with possible positive or negative sentiment, however we simply throw out such example out of our 
        population
    